In [6]:
from gurobipy import *
import numpy as np
import pandas as pd
from Hourbit import *
from Fibit import *
from blockbit import *


In [7]:
Blockbit()[0]

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1107,1,5,B,242.0,582.46,16,NaN
1,1104,1,4,B,1.0,663.68,10,NaN
2,1103,1,18,B,682.0,701.05,7,NaN
3,1101,1,15,B,320.0,490.82,8,NaN
4,1100,1,12,B,628.0,815.58,6,NaN
...,...,...,...,...,...,...,...,...
110,1095,1,16,B,72.0,532.01,8,NaN
111,1094,1,6,B,577.0,637.01,19,NaN
112,1093,1,17,B,477.0,604.37,3,NaN
113,1092,1,11,B,604.0,624.67,11,NaN


In [8]:
E = Model("Energy")
E.params.NonConvex =2
E.update()
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)
H = np.arange(1,25)

Set parameter NonConvex to value 2


In [9]:
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_D')
w_S = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_S')

In [10]:
Data_H_D=HourBit()[1]
Data_H_SP = HourBit()[0]

In [11]:
E.setObjective(sum(0.5*(2*Pair(j,h,Data_H_D,'DP')[0]+xD[h,j]*(Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]))*xD[h,j]*(Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(j,h,Data_H_SP,'SP')[0]+xS[h,j]*(Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]))*xS[h,j]*(Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])for j in J for h in H)
,GRB.MAXIMIZE)

In [12]:
E.addConstr(-sum((Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0])*xD[h,j] for j in J for h in H) +sum((Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])*xS[h,j] for j in J for h in H) -sum(Pair(1,h,Data_H_D,'DQ')[0] for h in H)+sum(Pair(1,h,Data_H_SP,'SQ')[0] for h in H) == 0)
# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)+1) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)+1) for h in H)
# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,j]<=xS[h,j] for j in range(2,find_largest(HIndex_S)+1) for h in H)
E.addConstrs(xS[h,j]<=w_S[h,j-1] for j in range(2,find_largest(HIndex_S)+1) for h in H)


{(2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (2, 20): <gurobi.Constr *Awaiting Model

In [13]:
E.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 95905 rows, 95904 columns and 239712 nonzeros
Model fingerprint: 0xe7b2cb99
Model has 47952 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+02, 4e+05]
  QObjective range [2e-02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Presolve removed 48000 rows and 47952 columns
Presolve time: 0.10s
Presolved: 47905 rows, 47952 columns, 143760 nonzeros
Presolved model has 47952 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.576e+04
 Factor NZ  : 8.089e+05 (roughly 45 MB of memory)
 Factor Ops : 1.703e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.38720267e+11  2.73299975e+10  

In [14]:
Res_D = []
Res_S = []
for i in range(1,1000):

    Res_D.append(E.getAttr('x',xD)[(2,i)])
    Res_S.append(E.getAttr('x',xS)[(2,i)])


for i in range(len(Res_D)):
    if Res_D[i] >= 1e-8 and Res_D[i] <= 0.999:
        print(i+1)
for i in range(len(Res_S)):
    if Res_S[i] >= 1e-8 and Res_S[i] <= 0.999:
        print(i+1)

285
284
